# Aspect Based Sentiment Analysis

- This Notebook Contains out Bi-LSTM Based Sentiment Classification Model + Other LSTM + Attention Based Models we experimented with.

#### References:
- https://github.com/mjain72/Sentiment-Analysis-using-Word2Vec-and-LSTM/blob/master/SentimentAnalysisTwitter.py
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/Attention
- https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
- https://github.com/rohit-gupta/POS-Attention/blob/master/pos-attention-model.py

""" AIT726 Final project - Part 2 - Polarity Classification using Deep learning models on SemEval’16 dataset ( 1708 training dataset and 587 testing dataset ) and Foursquare ( 849 testing dataset )

Authors: Yasas, Prashanti, Ashwini

Command to run the file: run DL_MODEL_PC.ipynb
"""

In [0]:
import sys

sys.path.append('/content/project')

In [0]:
%matplotlib inline

## Setup Requirements

- Load Libraries and Required Resources

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import cross_validate, KFold, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.metrics import confusion_matrix, roc_curve,  roc_auc_score, classification_report, f1_score, precision_score, recall_score
from sklearn.svm import SVC
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Attention, Bidirectional
from tensorflow.keras.layers import Lambda, dot, Activation, concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import Callback
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
import nltk
from nltk.corpus import stopwords
# Load From Project
from absa.config import DATA_PATHS
from absa.dataset import load_dataset

In [0]:
#Pass sentences through spacy nlp pipeline and get the output terms
nlp = spacy.load('en')

nltk.download('stopwords')
#Create a set of stopwords
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# 
create word2vec (  pre trained word embeddings ) from 
GoogleNews-vectors-negative300.bin.gz

In [0]:
word2vec = KeyedVectors.load_word2vec_format('/content/resources/GoogleNews-vectors-negative300.bin', binary=True)

'Word2Vec Vector Size: %d' % word2vec.vector_size

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


'Word2Vec Vector Size: 300'

    """Load and display semeval16 training dataset
       Load - Load dataset using load_dataset method ( Reads formatted XML file from the provided path )"""

In [0]:
train_ds_path = DATA_PATHS['asba.semeval16.raw.train']

df_train = load_dataset(train_ds_path)

df_train = df_train.loc[:, ['id', 'text', 'category', 'polarity']]

df_train = pd.DataFrame({
    'polarity': df_train.groupby(['id', 'text', 'category'])['polarity'].apply(list),
}).reset_index()

df_train.head()

,id,text,category,polarity
0,1004293:0,Judging from previous posts this used to be a ...,RESTAURANT#GENERAL,[negative]
1,1004293:1,"We, there were four of us, arrived at noon - t...",SERVICE#GENERAL,[negative]
2,1004293:2,"They never brought us complimentary noodles, i...",SERVICE#GENERAL,[negative]
3,1004293:3,The food was lousy - too sweet or too salty an...,FOOD#QUALITY,[negative]
4,1004293:3,The food was lousy - too sweet or too salty an...,FOOD#STYLE_OPTIONS,[negative]


    """Load and display semeval16 testing dataset
       Load - Load dataset using load_dataset method ( Reads formatted XML file from the provided path )"""

In [0]:
test_ds_path = DATA_PATHS['asba.semeval16.raw.test.gold']

df_test = load_dataset(test_ds_path)

df_test = df_test.loc[:, ['id', 'text', 'category', 'polarity']]

df_test = pd.DataFrame({
    'polarity': df_test.groupby(['id', 'text', 'category'])['polarity'].apply(list),
}).reset_index()

df_test.head()

,id,text,category,polarity
0,en_BlueRibbonSushi_478218171:0,Yum!,FOOD#QUALITY,[positive]
1,en_BlueRibbonSushi_478218171:1,Serves really good sushi.,FOOD#QUALITY,[positive]
2,en_BlueRibbonSushi_478218171:2,Not the biggest portions but adequate.,FOOD#STYLE_OPTIONS,[neutral]
3,en_BlueRibbonSushi_478218171:3,Green Tea creme brulee is a must!,FOOD#QUALITY,[positive]
4,en_BlueRibbonSushi_478218171:4,Don't leave the restaurant without it.,FOOD#QUALITY,[positive]


    """Load and display Foursquare testing dataset
       Load - Load dataset using load_dataset method ( Reads formatted XML file from the provided path )"""

In [0]:
test_fs_ds_path = DATA_PATHS['asba.foursquare.raw.test.gold']

df_test_fs = load_dataset(test_fs_ds_path)

df_test_fs = df_test_fs.loc[:, ['id', 'text', 'category', 'polarity']]

df_test_fs = pd.DataFrame({
    'polarity': df_test_fs.groupby(['id', 'text', 'category'])['polarity'].apply(list),
}).reset_index()

df_test_fs.head()

,id,text,category,polarity
0,0:0,2 words -filter coffee :-),DRINKS#QUALITY,[negative]
1,101:0,Absolute favourite hotpot in town!,RESTAURANT#GENERAL,[positive]
2,102:0,"Great great food, not too pricey, good service...",FOOD#QUALITY,[positive]
3,102:0,"Great great food, not too pricey, good service...",LOCATION#GENERAL,[positive]
4,102:0,"Great great food, not too pricey, good service...",RESTAURANT#PRICES,[positive]


"""As we have multiple aspects we have used MultiLabelBinarizer to create y_train, y_test and y_test_fs"""


In [0]:
mlb = MultiLabelBinarizer()

y_train = mlb.fit_transform(df_train.polarity)

y_test = mlb.transform(df_test.polarity)

y_test_fs = mlb.transform(df_test_fs.polarity)

y_train.shape, y_test.shape, y_test_fs.shape

((2258, 3), (743, 3), (1034, 3))


""" Preprocess features - Train and test data , pos are passed through spacy_doc pipeline to generate the list of tokens, which are later passed through keras tokenizer to convert them to integers. Finally they are padded to maxlenght . By performing above steps we have our x_train, x_test and x_test_fs input features """

In [0]:
maxlen = 50

train_tokens = []
train_pos_tags = []

for sent in df_train.text:
  spacy_doc = nlp(sent)
  train_tokens.append([t.text for t in spacy_doc])
  train_pos_tags.append([t.pos_ for t in spacy_doc])

test_tokens = []
test_pos_tags = []

for sent in df_test.text:
  spacy_doc = nlp(sent)
  test_tokens.append([t.text for t in spacy_doc])
  test_pos_tags.append([t.pos_ for t in spacy_doc])

test_fs_tokens = []
test_fs_pos_tags = []

for sent in df_test_fs.text:
  spacy_doc = nlp(sent)
  test_fs_tokens.append([t.text for t in spacy_doc])
  test_fs_pos_tags.append([t.pos_ for t in spacy_doc])

#Convert words to integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_tokens + test_tokens + test_fs_tokens)
x_train = tokenizer.texts_to_sequences(train_tokens)
x_test = tokenizer.texts_to_sequences(test_tokens)
x_test_fs = tokenizer.texts_to_sequences(test_fs_tokens)

#Convert POS to integers
pos_tokenizer = Tokenizer()
pos_tokenizer.fit_on_texts(train_pos_tags + test_pos_tags + test_fs_pos_tags)
x_pos_train = pos_tokenizer.texts_to_sequences(train_pos_tags)
x_pos_test = pos_tokenizer.texts_to_sequences(test_pos_tags)
x_pos_test_fs = pos_tokenizer.texts_to_sequences(test_fs_pos_tags)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_pos_train = pad_sequences(x_pos_train, maxlen=maxlen)

x_test = pad_sequences(x_test, maxlen=maxlen)
x_pos_test = pad_sequences(x_pos_test, maxlen=maxlen)

x_test_fs = pad_sequences(x_test_fs, maxlen=maxlen)
x_pos_test_fs = pad_sequences(x_pos_test_fs, maxlen=maxlen)

x_train.shape, x_test.shape, x_test_fs.shape

((2258, 50), (743, 50), (1034, 50))

create aspect_encoder using LabelBinarizer() to be used later in the model

In [0]:
aspect_encoder = LabelBinarizer()

x_train_aspect = aspect_encoder.fit_transform(df_train.category)
x_test_aspect = aspect_encoder.transform(df_test.category)
x_test_fs_aspect = aspect_encoder.transform(df_test_fs.category)

x_train_aspect.shape, x_test_aspect.shape, x_test_fs_aspect.shape

((2258, 12), (743, 12), (1034, 12))

## Create Model

""" create word embeddings and embedding_index based on word2vec """

In [0]:
word_index = tokenizer.word_index

seq_length = maxlen
embedding_dim = word2vec.syn0.shape[1]
vocab_size = len(word_index) + 1

embeddings_index = word2vec

embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
  if word in embeddings_index:
    embedding_vector = embeddings_index[word]
    embedding_matrix[i] = embedding_vector
  else:
      pass # for words not in embedding index

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  after removing the cwd from sys.path.


In [0]:
#Extract vocabulary size and word embedding dimension
max_tokens, dimension = len(pos_tokenizer.word_index)+1, 100

""" Metrics on_train_begin and on_epoch_end are callback functions used to print intermediate results during training"""

In [0]:
class Metrics(Callback):
  def __init__(self, model, validation_data):
    self.model_ = model
    self.validation_data = validation_data

  def on_train_begin(self, logs={}):
    print('epoch, precision_micro, recall_micro, f1_micro')
  
  def on_epoch_end(self, epoch, logs={}):
    val_predict = (np.asarray(self.model_.predict(self.validation_data[0]))).round()
    val_targ = self.validation_data[1]
    _val_precision = precision_score(val_targ, val_predict, average='micro')
    _val_recall = recall_score(val_targ, val_predict, average='micro')
    _val_f1 = f1_score(val_targ, val_predict, average='micro')
    print('%d, %f, %f, %f' % (epoch + 1, _val_precision, _val_recall, _val_f1))

""" Attention layer is created , reference to the source code usage has been provided """

In [0]:
# from https://github.com/philipperemy/keras-attention-mechanism/blob/master/attention/attention.py

def attention_3d_block(hidden_states, output_size=32):
    """
    Many-to-one attention mechanism for Keras.
    @param hidden_states: 3D tensor with shape (batch_size, time_steps, input_dim).
    @return: 2D tensor with shape (batch_size, 64)
    @author: felixhao28.
    """
    hidden_size = int(hidden_states.shape[2])
    # Inside dense layer
    #              hidden_states            dot               W            =>           score_first_part
    # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
    # W is the trainable weight matrix of attention Luong's multiplicative style score
    score_first_part = Dense(hidden_size, use_bias=False, name='attention_score_vec')(hidden_states)
    #            score_first_part           dot        last_hidden_state     => attention_weights
    # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
    h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(hidden_states)
    score = dot([score_first_part, h_t], [2, 1], name='attention_score')
    attention_weights = Activation('softmax', name='attention_weight')(score)
    # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
    context_vector = dot([hidden_states, attention_weights], [1, 1], name='context_vector')
    pre_activation = concatenate([context_vector, h_t], name='attention_output')
    attention_vector = Dense(output_size, use_bias=False, activation='tanh', name='attention_vector')(pre_activation)
    return attention_vector

"""create_model - Create different models such as 'LSTM', 'self-attention', 'pos-attention', 'bidirectional. For all the models we have one layer of 64 hidden units, and a fully connected output layer using sigmoid as activation function. We have used Adam optimizer, and cross-entropy for the loss function with learning rate 0.001 for all the models.
In addition to aspect classification, here in polarity classification we are passing aspect dimention and aspect encoder to the concatenation layer"""

In [0]:
aspects_dim = 12
lstm_dropout = 0.8
lstm_out = 64
use_model =  ['LSTM', 'self-attention', 'pos-attention', 'bidirectional'][3]
loss = 'binary_crossentropy'
def create_model():
  optimizer = optimizers.Adam(learning_rate=0.0001)
  embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=seq_length)

  # Word Embedding
  word_input_layer = Input(shape=(seq_length,))
  word_embedding_output = embedding_layer(word_input_layer)

  # POS Embedding
  if use_model == 'pos-attention':
    pos_embedding_layer = Embedding(max_tokens, dimension, input_length=seq_length)
    pos_input_layer = Input(shape=(seq_length,), name='pos_seq')
    pos_embedding_output = pos_embedding_layer(pos_input_layer)

  # word_pos_attention_seq = Attention()([word_embedding_output, pos_embedding_output])

  if use_model == 'self-attention':
    lstm_output = LSTM(units=lstm_out, dropout=lstm_dropout, return_sequences=True)(word_embedding_output)
    lstm_output = attention_3d_block(lstm_output)
  elif use_model == 'pos-attention':
    pass
  elif use_model == 'bidirectional':
    lstm_output = Bidirectional(LSTM(units=lstm_out, dropout=lstm_dropout))(word_embedding_output)
  else:
    lstm_output = LSTM(units=lstm_out, dropout=lstm_dropout)(word_embedding_output)

  aspect_input_layer = Input(shape=(aspects_dim, ))

  features = concatenate([lstm_output, aspect_input_layer])
  # features = lstm_output

  output_layer = Dense(3, activation='sigmoid')(features)

  model = Model(inputs=[word_input_layer, aspect_input_layer], outputs=output_layer)

  model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

  return model

model = create_model()

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 300)      1341900     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 128)          186880      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 12)]         0                                            
______________________________________________________________________________________________

## Cross Validate

"""Perform 5-fold cross validation on the training dataset and use Metrics to derive intermediate evaluation results and finally we are using these results we perform error analysis """

In [0]:
# Five fold Cross-Validation 
if False: # Done
  y_preds = np.zeros(y.shape)
  kf = KFold(n_splits=5)
  print(model.summary())
  for i, (train_index, test_index) in enumerate(kf.split(X)):
    model, metrics = None, None
    print('Epoch: %d' % i)
    X_aspect_train, X_aspect_test = X_aspect[train_index], X_aspect[test_index]
    X_train, X_test, Y_train, Y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    model = create_model()
    metrics = Metrics(model, validation_data=((X_test, X_aspect_test), Y_test))
    model.fit((X_train, X_aspect_train), Y_train, epochs=num_epochs, verbose=0, batch_size=batch_size, callbacks=[metrics])
    # predict the results
    score, acc = model.evaluate((X_test, X_aspect_test), Y_test, verbose=2, batch_size=batch_size)
    y_pred = model.predict((X_test, X_aspect_test))
    y_preds[test_index] = y_pred
    y_pred_bool = y_pred > threshold
    # ROC AUC curve
    roc_auc = roc_auc_score(Y_test, y_pred)
    df = evaluate(Y_test, y_pred_bool)
    print(df.to_csv())
    print("roc_auc, {}".format(roc_auc))
    df_train['predictions'] = [list(x) for x in mlb.inverse_transform(y_preds > 0.5)]
    df_train.to_excel('.project/output/preds/pc_dl.xlsx')



```
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_7 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 50, 300)      931500      input_7[0][0]                    
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 128)          186880      embedding_3[0][0]                
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 12)]         0                                            
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 140)          0           bidirectional_3[0][0]            
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 3)            423         concatenate_3[0][0]              
==================================================================================================
Total params: 1,118,803
Trainable params: 1,118,803
Non-trainable params: 0
__________________________________________________________________________________________________
None
Epoch: 0
epoch, precision_micro, recall_micro, f1_micro
1, 0.780269, 0.746781, 0.763158
2, 0.747727, 0.706009, 0.726269
3, 0.775281, 0.740343, 0.757409
4, 0.776018, 0.736052, 0.755507
5, 0.803167, 0.761803, 0.781938
6, 0.818391, 0.763948, 0.790233
7, 0.816964, 0.785408, 0.800875
8, 0.814220, 0.761803, 0.787140
9, 0.820046, 0.772532, 0.795580
10, 0.842593, 0.781116, 0.810690
11, 0.829885, 0.774678, 0.801332
12, 0.834483, 0.778970, 0.805771
13, 0.830275, 0.776824, 0.802661
14, 0.841232, 0.761803, 0.799550
15, 0.825287, 0.770386, 0.796892
16, 0.827354, 0.791845, 0.809211
17, 0.831081, 0.791845, 0.810989
18, 0.816372, 0.791845, 0.803922
19, 0.811816, 0.796137, 0.803900
20, 0.815385, 0.796137, 0.805646
15/15 - 0s - loss: 0.3440 - accuracy: 0.8031
index,value
accuracy,0.7522123893805309
precision_macro,0.7444842994536663
precision_micro,0.8153846153846154
recall_macro,0.5672030651340996
recall_micro,0.796137339055794
f1_macro,0.5931287581186977
f1_micro,0.8056460369163952

roc_auc, 0.879261813802623
Epoch: 1
epoch, precision_micro, recall_micro, f1_micro
1, 0.713592, 0.639130, 0.674312
2, 0.793333, 0.776087, 0.784615
3, 0.821918, 0.782609, 0.801782
4, 0.819413, 0.789130, 0.803987
5, 0.807606, 0.784783, 0.796031
6, 0.837104, 0.804348, 0.820399
7, 0.827586, 0.782609, 0.804469
8, 0.847575, 0.797826, 0.821948
9, 0.857477, 0.797826, 0.826577
10, 0.869464, 0.810870, 0.839145
11, 0.844749, 0.804348, 0.824053
12, 0.851163, 0.795652, 0.822472
13, 0.842105, 0.800000, 0.820513
14, 0.848131, 0.789130, 0.817568
15, 0.839161, 0.782609, 0.809899
16, 0.831019, 0.780435, 0.804933
17, 0.837104, 0.804348, 0.820399
18, 0.819820, 0.791304, 0.805310
19, 0.822616, 0.806522, 0.814490
20, 0.817787, 0.819565, 0.818675
15/15 - 0s - loss: 0.3358 - accuracy: 0.8053
index,value
accuracy,0.7676991150442478
precision_macro,0.5921877736906639
precision_micro,0.8177874186550976
recall_macro,0.5438344159191669
recall_micro,0.8195652173913044
f1_macro,0.5518709320038776
f1_micro,0.8186753528773072

roc_auc, 0.8397482100147294
Epoch: 2
epoch, precision_micro, recall_micro, f1_micro
1, 0.750000, 0.712418, 0.730726
2, 0.802850, 0.736383, 0.768182
3, 0.826698, 0.769063, 0.796840
4, 0.841379, 0.797386, 0.818792
5, 0.850117, 0.790850, 0.819413
6, 0.822472, 0.797386, 0.809735
7, 0.853547, 0.812636, 0.832589
8, 0.833708, 0.808279, 0.820796
9, 0.863962, 0.788671, 0.824601
10, 0.846868, 0.795207, 0.820225
11, 0.842956, 0.795207, 0.818386
12, 0.837647, 0.775599, 0.805430
13, 0.823144, 0.821351, 0.822246
14, 0.823400, 0.812636, 0.817982
15, 0.825000, 0.790850, 0.807564
16, 0.828571, 0.821351, 0.824945
17, 0.838269, 0.801743, 0.819599
18, 0.812775, 0.803922, 0.808324
19, 0.846330, 0.803922, 0.824581
20, 0.832215, 0.810458, 0.821192
15/15 - 0s - loss: 0.3165 - accuracy: 0.8097
index,value
accuracy,0.7920353982300885
precision_macro,0.690357023690357
precision_micro,0.8322147651006712
recall_macro,0.576789501590668
recall_micro,0.8104575163398693
f1_macro,0.6096637973940336
f1_micro,0.8211920529801325

roc_auc, 0.858959524583201
Epoch: 3
epoch, precision_micro, recall_micro, f1_micro
1, 0.618056, 0.588106, 0.602709
2, 0.778291, 0.742291, 0.759865
3, 0.773543, 0.759912, 0.766667
4, 0.783599, 0.757709, 0.770437
5, 0.806897, 0.773128, 0.789651
6, 0.757720, 0.702643, 0.729143
7, 0.815909, 0.790749, 0.803132
8, 0.784922, 0.779736, 0.782320
9, 0.762980, 0.744493, 0.753623
10, 0.778742, 0.790749, 0.784699
11, 0.789011, 0.790749, 0.789879
12, 0.738046, 0.781938, 0.759358
13, 0.780911, 0.792952, 0.786885
14, 0.765823, 0.799559, 0.782328
15, 0.779221, 0.792952, 0.786026
16, 0.767033, 0.768722, 0.767877
17, 0.777538, 0.792952, 0.785169
18, 0.748963, 0.795154, 0.771368
19, 0.752643, 0.784141, 0.768069
20, 0.744283, 0.788546, 0.765775
15/15 - 0s - loss: 0.5663 - accuracy: 0.7561
index,value
accuracy,0.7117516629711752
precision_macro,0.6655625356938892
precision_micro,0.7442827442827443
recall_macro,0.5539475388821841
recall_micro,0.788546255506608
f1_macro,0.5457204991456531
f1_micro,0.7657754010695188

roc_auc, 0.8324862898171297
Epoch: 4
epoch, precision_micro, recall_micro, f1_micro
1, 0.592018, 0.579176, 0.585526
2, 0.724706, 0.668113, 0.695260
3, 0.776744, 0.724512, 0.749719
4, 0.710046, 0.674620, 0.691880
5, 0.767654, 0.731020, 0.748889
6, 0.767654, 0.731020, 0.748889
7, 0.768879, 0.728850, 0.748330
8, 0.775463, 0.726681, 0.750280
9, 0.778037, 0.722343, 0.749156
10, 0.798561, 0.722343, 0.758542
11, 0.757075, 0.696312, 0.725424
12, 0.770089, 0.748373, 0.759076
13, 0.780822, 0.741866, 0.760845
14, 0.762749, 0.746204, 0.754386
15, 0.764302, 0.724512, 0.743875
16, 0.768722, 0.757050, 0.762842
17, 0.761161, 0.739696, 0.750275
18, 0.763393, 0.741866, 0.752475
19, 0.764192, 0.759219, 0.761697
20, 0.750000, 0.767896, 0.758842
15/15 - 0s - loss: 0.4530 - accuracy: 0.7517
index,value
accuracy,0.7028824833702882
precision_macro,0.6144501367542726
precision_micro,0.75
recall_macro,0.5376422493670555
recall_micro,0.7678958785249458
f1_macro,0.532139823309548
f1_micro,0.7588424437299035

roc_auc, 0.8383861659676043
```



## Train Model

In [ ]:
#Train the model on with validation data
x_train_1, x_valid, x_train_aspect_1, x_valid_aspect, y_train_1, y_valid = train_test_split(x_train, x_train_aspect, y_train, test_size= 0.33, random_state = 24)

In [0]:
#fit model
batch_size = 32
num_epochs = 100
# metrics = Metrics(model, validation_data=((x_valid, x_valid_aspect), y_valid))

model.fit((x_train_1, x_train_aspect_1), y_train_1, epochs=num_epochs, batch_size=batch_size, validation_data=((x_valid, x_valid_aspect), y_valid))

""" Fit model on the complete training data set """"

In [0]:
#fit model
batch_size = 32
num_epochs = 7

model.fit((x_train, x_train_aspect), y_train, epochs=num_epochs, verbose=1, batch_size=batch_size)

Epoch 1/7
71/71 [==============================] - 1s 16ms/step - loss: 0.6435 - accuracy: 0.5642
Epoch 2/7
71/71 [==============================] - 1s 16ms/step - loss: 0.4917 - accuracy: 0.6466
Epoch 3/7
71/71 [==============================] - 1s 16ms/step - loss: 0.4707 - accuracy: 0.6479
Epoch 4/7
71/71 [==============================] - 1s 16ms/step - loss: 0.4593 - accuracy: 0.6585
Epoch 5/7
71/71 [==============================] - 1s 16ms/step - loss: 0.4338 - accuracy: 0.6864
Epoch 6/7
71/71 [==============================] - 1s 17ms/step - loss: 0.3880 - accuracy: 0.7378
Epoch 7/7
71/71 [==============================] - 1s 17ms/step - loss: 0.3702 - accuracy: 0.7595


## Evaluate on the test data 

In [0]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 300)      1341900     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 128)          186880      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 12)]         0                                            
______________________________________________________________________________________________

### SemEval-16 Test Data

""" Model predict on SemEval-16 test dataset"""

In [0]:
threshold = 0.5

# predict the results
score, acc = model.evaluate((x_test, x_test_aspect), y_test, verbose=2, batch_size=batch_size)
y_pred = model.predict((x_test, x_test_aspect))
y_pred_bool = y_pred > threshold
# ROC AUC curve
roc_auc = roc_auc_score(y_test, y_pred)
f1_micro = f1_score(y_test, y_pred_bool, average='micro')

print("roc_auc, {}".format(roc_auc))

report = classification_report(y_test, y_pred_bool, target_names=list(mlb.classes_), output_dict=True)

df = pd.DataFrame(report).transpose()

print(df.to_csv())

24/24 - 0s - loss: 0.3399 - accuracy: 0.8116
roc_auc, 0.8753558382365396
,precision,recall,f1-score,support
negative,0.6757990867579908,0.7589743589743589,0.714975845410628,195.0
neutral,0.625,0.11904761904761904,0.19999999999999998,42.0
positive,0.8714555765595463,0.896887159533074,0.8839884947267498,514.0
micro avg,0.8121693121693122,0.8175765645805593,0.8148639681486397,751.0
macro avg,0.7240848877725123,0.5916363791850173,0.5996547800457925,751.0
weighted avg,0.8068694917036152,0.8175765645805593,0.8018513663710011,751.0
samples avg,0.8088829071332436,0.819650067294751,0.8111260655002244,751.0



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Foursquare Review Data

""" Model predict on test Foursqure dataset"""

In [0]:
threshold = 0.5

# predict the results
y_pred_fs = model.predict((x_test_fs, x_test_fs_aspect))
y_pred_fs_bool = y_pred_fs > threshold
# ROC AUC curve
# roc_auc = roc_auc_score(y_test_fs, y_pred_fs)
# print("roc_auc, {}".format(roc_auc))

report = classification_report(y_test_fs, y_pred_fs_bool, target_names=list(mlb.classes_), output_dict=True)

df = pd.DataFrame(report).transpose()

print(df.to_csv())

,precision,recall,f1-score,support
negative,0.6229508196721312,0.6229508196721312,0.6229508196721312,183.0
neutral,0.3333333333333333,0.16666666666666666,0.2222222222222222,18.0
positive,0.9301204819277108,0.9146919431279621,0.9223416965352449,844.0
micro avg,0.8698630136986302,0.8507177033492823,0.8601838413159167,1045.0
macro avg,0.628801544977725,0.5681031431555866,0.5891715794765328,1045.0
weighted avg,0.8660494610019024,0.8507177033492823,0.8578530065796619,1045.0
samples avg,0.8515473887814313,0.8544487427466151,0.8507414571244358,1045.0



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Save the Model

""" Save the required model, tokenizer and mlb for demo purposes """

In [0]:
import pickle

model.save('/content/project/output/models/dl_model_pc.h5')

# saving tokenizer
with open('/content/project/output/models/tokenizer_pc.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# saving mlb
with open('/content/project/output/models/mlb_pc.pickle', 'wb') as handle:
    pickle.dump(mlb, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# saving aspect encoder
np.save('/content/project/output/models/aspect_enc_classes.npy', aspect_encoder.classes_)